# Cohort Selection Using MIDRC Temporal COVID Test Data
---
by Chris Meyer, PhD

Manager of Data and User Services at the Center for Translational Data Science at the University of Chicago

August 2022

---
This Jupyter notebook tutorial demonstrates how to use the MIDRC data commons' APIs to access imaging study and COVID-19 test data, how to use temporal properties in those data to select a cohort of COVID-19 positive imaging studies, and how to access those image files.

# Python packages:

In [ ]:
# The packages below may be necessary for users to install according to the imports necessary in the subsequent cells.

#!pip install --upgrade pandas
#!pip install --upgrade --ignore-installed PyYAML
#!pip install --upgrade pip
#!pip install --upgrade gen3 --user --upgrade
#!pip install cdiserrors
#!pip install --upgrade pydicom

# Import Python Packages and scripts

In [ ]:
# Import Python Packages and scripts
import pandas as pd
import sys, os, webbrowser
import gen3
import pydicom
import matplotlib.pyplot as plt

from gen3.submission import Gen3Submission
from gen3.auth import Gen3Auth
from gen3.index import Gen3Index
from gen3.query import Gen3Query

In [ ]:
# Import some custom Python scripts from personal GitHub repo
# change these directory paths to reflect your local working directory

home_dir = "/Users/christopher" 
demo_dir = "{}/Documents/Notes/MIDRC/tutorials".format(home_dir)

os.chdir(demo_dir)

os.system("wget https://raw.githubusercontent.com/cgmeyer/gen3sdk-python/master/expansion/expansion.py -O {}/expansion.py".format(demo_dir))
%run expansion.py


In [ ]:
# Initiate instances of the Gen3 SDK Classes using credentials file for authentication
# Change the directory path in "cred" to reflect the location of your credentials file.

api = "https://data.midrc.org"
cred = "{}/Downloads/midrc-credentials.json".format(home_dir)
auth = Gen3Auth(api, refresh_file=cred) # authentication class
sub = Gen3Submission(api, auth) # submission class
query = Gen3Query(auth) # query class
exp = Gen3Expansion(api,auth,sub) # class with some custom scripts
exp.get_project_ids()



## How can one associate the date of an imaging exam with the date of COVID-19 test results for a patient?
---

Specific dates are not allowed in the MIDRC data commons, but given a single "index_event" for a case, "days to X from index event" properties are provided.

For example, one can query or export the imaging_study node, which has "days_to_study", and the measurement node, which has "test_days_from_index", and merge into a single table on "case_ids" (the unique, de-identified patient identifiers) to create a temporal timeline of imaging studies and COVID-19 tests for a cohort of patients.


## Export metadata using submission API
---
Here we'll utilize the MIDRC submission API to export all the imaging study and measurement (COVID-19 tests) data using the ["get_node_tsvs" function](https://github.com/cgmeyer/gen3sdk-python/blob/2aecc6575b22f9cca279b650914971dd6723a2ce/expansion/expansion.py#L219), which is a wrapper to export and merge all the records in a node across each project in the data commons using the [Gen3SDK](https://github.com/uc-cdis/gen3sdk-python/) function [Gen3Submission.export_node()](https://github.com/uc-cdis/gen3sdk-python/blob/5d7b5270ff11cf7037f211cf01e410d8e73d6b84/gen3/submission.py#L361).

In [ ]:
# Export all the records in the imaging_study node
st = exp.get_node_tsvs(node='imaging_study')
print('\nrows:{}, columns:{}'.format(st.shape[0],st.shape[1]))

In [ ]:
# Filter the imaging_study data for only studies that have a non-null "days_to_study" and "DX" study_modality
s = st.loc[(~st['days_to_study'].isna()) & (st['study_modality']=='DX')]
print('rows:{}, columns:{}'.format(s.shape[0],s.shape[1]))
s.head(3)

In [ ]:
#Export all the data in the measurement node, which is used to store the COVID test data
meas = exp.get_node_tsvs(node='measurement')
print('\nrows:{}, columns:{}'.format(meas.shape[0],meas.shape[1]))
meas.head(3)

In [ ]:
## Filter the measurements for only COVID-19 tests with a non-null "test_days_from_index" property
m = meas.loc[(~meas['test_days_from_index'].isna()) & (meas['test_name']=='COVID-19')]
print('\nrows:{}, columns:{}'.format(m.shape[0],m.shape[1]))
m.head(3)

In [ ]:
## Check out the properties in each DataFrame to help make a list of properties to merge into a single table
display(list(s))
display(len(s))
display(list(m))
display(len(m))

In [ ]:
## Merge the imaging_study and measurement data using "case_ids" as a foreign key
temp = pd.merge(s[['study_uid','days_to_study','case_ids']],m[['project_id','submitter_id','test_name','test_result_text','case_ids','test_days_from_index']],on='case_ids')
print('\nrows:{}, columns:{}'.format(temp.shape[0],temp.shape[1]))
display(temp)

## Calculate the days from COVID-19 test to an imaging_study
---
Now that we have the temporal data for imaging studies and COVID-19 tests in a single DataFrame for all cases in MIDRC for which this data is provided, we can calculate the number of days between each imaging study and each COVID-19 test, which we'll call `days_from_study_to_test`.

* Note: In MIDRC, a negative "days to XYZ" indicates that the event XYZ took place that many days prior to the index event, while a positive "days to" indicates the number of days since the index event. For example, a "days_to_study" of "-10" indicates that the imaging study was performed 10 days *before* the index event. A value of "365" indicates the imaging study took place one year *after* the index event. 

In the case of a derived property like `days_from_study_to_test`, the date of the study can be thought of as the 0 point, and the test takes place in time either before the study, moving backwards on the timeline (negative value) or the test takes place after the study (moving forward in time).

So, we expect a positive value for `days_from_study_to_test` if the test was performed after the study.
- For example, if `test_days_from_index` is `1` and `days_to_study` is `4`, the `days_from_study_to_test` should be `-3`, which means the test took place 3 days before the study.
- If the COVID test is on day 4 and the imaging study is on day 1, then the `days_from_study_to_test` is `3`, meaning the COVID-19 test took place 3 days after the imaging study.



In [ ]:
## Calculate the days from COVID-19 test to an imaging_study
temp['days_from_study_to_test'] = temp['test_days_from_index'] - temp['days_to_study']
display(temp)

## Identify "COVID-19 positive" imaging studies
---
Now that we've calculated `days_from_study_to_test`, we can define a cut-off value and filter the imaging studies using that value to determine which imaging studies were performed within a certain time-frame of receiving a positive COVID-19 test.

Again, our new derived attribute `days_from_study_to_test` has a positive value if the COVID test was performed after the imaging study (i.e., from the study date to test date is moving forward in time) and a negative value if the COVID test was performed before the imaging study (i.e., go back in time from the imaging date to the COVID test date). 

For this demo, let's assume that an imaging study was performed when a person was "COVID-positive" if the imaging study was performed within a 7 day window after a positive test result. So, we'll filter the DataFrame of studies for a `days_from_study_to_test` in the range of -7 to 0 and also require the `test_result_text` to be `Positive`.


In [ ]:
ps = temp.loc[(temp['days_from_study_to_test'] < 0) & (temp['days_from_study_to_test'] > -7) & (temp['test_result_text']=='Positive')]
display(ps)

In [ ]:
#Saving the data frame to a csv
os.chdir(demo_dir)
filename = 'DX_imaging_studies_plus_covid_tests.tsv' 
ps.to_csv(filename,sep='\t',index=False)


## Get the imaging files for the identified studies or cases.
---
Now that we have a list of imaging studies that were deemed to take place soon after a patient was infected with COVID-19, we can use the study_uid, which is a unique identifier for imaging studies, to collect the associated files. 

Note: If we want *all* the imaging studies for the cohort of identified cases, e.g., to have a "healthy" or "baseline" images for comparison, we can instead use the case_ids to pull all imaging files for the cases, keeping in mind that this will pull any additional imaging studies that may fall outside our defined temporal range.

In [ ]:
## Make a list of study_uids and case_ids

## read in previously saved DataFrame if restarting notebook:
# pd.read_csv(filename, sep='\t', dtype=str)

cids = list(set(ps['case_ids']))
display(len(cids))

sids = list(set(ps['study_uid']))
display(len(sids))

In [ ]:
## This query retrieves ALL imaging_study records, we will next filter these results based on the COVID test data
data = query.raw_data_download(
    data_type="imaging_study",
    fields=[
        "study_uid",
        "case_ids",
        "object_id",
        "project_id"
    ],
    sort_fields=[{"study_uid": "asc"}],
    accessibility="accessible"
)

In [ ]:
## Take a glance at the returned data
display(len(data))
display(data[0])


In [ ]:
# convert the query data to a DataFrame and remove any records that lack a study_uid or object_id
studies = pd.DataFrame(data)
studies = studies.loc[(~studies['object_id'].isna())&(~studies['study_uid'].isna())]
display(len(studies))
studies.head()

In [ ]:
# Convert lists to strings; necessary because the properties case_ids and object_id are arrays in the dictionary, and thus are returned as lists.
studies['case_ids'] = [','.join(map(str, l)) for l in studies['case_ids']]

In [ ]:
# Now filter the imaging studies based on our temporal results
covid_studies = studies.loc[studies['study_uid'].isin(sids)]
len(covid_studies)

In [ ]:
# save our result to a csv
filename = "covid_positive_DX_imaging_studies_7d_window_with_object_ids.tsv"
covid_studies.to_csv(filename, sep='\t', index=False)


In [ ]:
object_ids = list(set([a for b in covid_studies.object_id.tolist() for a in b]))

In [ ]:
len(object_ids)

## Now that we have a list of file object_ids for the desired imaging studies, we can use the Gen3 SDK "drs-pull" commands to access the files themselves.
---
First, we'll create a manifest.json file using a [simple script](https://github.com/cgmeyer/gen3sdk-python/blob/389e3945482439ace6e4536e6d0e35c6e48de9c9/expansion/expansion.py#L2575). Then we'll use the `gen3 drs-pull manifest` command to download the files.

See the detailed documentation to learn more about the Gen3 SDK drs-pull command: https://github.com/uc-cdis/gen3sdk-python/blob/master/docs/howto/drsDownloading.md


In [ ]:
# Save the manifest of file object_ids to a JSON file
mani_name = 'MIDRC_DX_imaging_studies_covid_positive_manifest.json'
exp.write_manifest_for_guids(guids=object_ids, filename=mani_name)


In [ ]:
# To download all files in the manifest, use the "gen3 drs-pull manifest" command
download_dir = "{}/images".format(demo_dir)

if not os.path.exists(download_dir):
    os.makedirs(download_dir)
    
cmd = "gen3 --auth {} --endpoint data.midrc.org drs-pull manifest {} {}".format(cred, mani_name, download_dir)
print(cmd)


In [ ]:
## Run the manifest download command. 
## Note that this will take some time if the manifest is very large. It makes more sense to copy the above command and run in your terminal instead of from a Jupyter Notebook to monitor the progress in real-time.
# subprocess.run(cmd, shell=True, capture_output=True)


## Now download a single DX image and display it in the notebook
---
Now we'll download a single x-ray file using the `gen3 drs-pull object` command and display the image and it's embedded metadata on the screen.




In [ ]:
# Prepare to download a single image file via it's object_id using the gen3 SDK; save object_id to a variable
case_ids = covid_studies.iloc[0]['case_ids']
study_uid = covid_studies.iloc[0]['study_uid']
object_id = covid_studies.iloc[0]['object_id'][0]

display(case_ids)
display(study_uid)
display(object_id)

In [ ]:
# Build the SDK command to send to the shell.
# Note: "gen3" refers to a Gen3 SDK function that runs at the users command line
# Users may experience errors or warnings but may have still downloaded the file. Check this in your working directory.

cmd = "gen3 --auth {} --endpoint data.midrc.org drs-pull object {}".format(cred,object_id)
display(cmd)


In [ ]:
# Run the download command.
subprocess.run(cmd, shell=True, capture_output=True)

In [ ]:
# The above command should have successfully downloaded a new directory with a zipped file.  
cmd = "ls -l {}/{}".format(case_ids,study_uid)
stout = subprocess.run(cmd, shell=True, capture_output=True)
print(stout)

In [ ]:
## Grab the filename and series UID of the downloaded file using RegEx
import re

m = re.search(' ([0-9\.]+.zip)', str(stout))

if m:
    zip_file = m.group(1)
    print(zip_file)
else:
    print("No zip found.")

series_uid = re.sub("(\.zip)", "", zip_file)
print(series_uid)

In [ ]:
# Unzip the imaging series package
from zipfile import ZipFile

with ZipFile('{}/{}/{}/{}'.format(demo_dir,case_ids,study_uid,zip_file), 'r') as zipObj:
    zipObj.extractall()

In [ ]:
#Input the name of the newly create .dcm file
cmd = "ls -l {}/{}/{}".format(case_ids,study_uid,series_uid)
stout = subprocess.run(cmd, shell=True, capture_output=True)
print(stout)


In [ ]:
# Get the name of the first DICOM file in the extracted imaging series
m = re.search(' ([0-9\.]+.dcm)', str(stout))

if m:
    dcm_file = m.group(1)
    print(dcm_file)
else:
    print("No DCM files found.")


In [ ]:
# Read in the DCM file using the python DICOM package pydicom
dimg = pydicom.dcmread("{}/{}/{}/{}".format(case_ids,study_uid,series_uid,dcm_file),force=True)


In [ ]:
dimg

In [ ]:
## tell matplotlib to display our images as 6 x 6 inch image, with resolution of 100 dpi
plt.figure(figsize = (6,6), dpi=100) 

## tell matplotlib to display our image, using a gray-scale lookup table.
plt.imshow(dimg.pixel_array, cmap=plt.cm.gray)